## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Dec. 15th Dec. 22nd


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,December-19-2024,Condo/Co-op,68 SE 6th St Ph 4303,Miami,FL,33131.0,5800000.0,4.0,4.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/68-SE-6th-St-3...,MARMLS,A11711930,N,Y,25.768232,-80.192374


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
77,PAST SALE,2024-12-23,Condo/Co-op,9310 W Flagler St Unit 110A,Miami,FL,33174.0,220000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/9310-W-Flagler...,MARMLS,A11641348,N,Y,25.767336,-80.347122
134,PAST SALE,2024-12-23,Condo/Co-op,10185 Collins Ave #1409,Bal Harbour,FL,33154.0,720000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10185-Co...,MARMLS,A11569164,N,Y,25.894433,-80.123184
27,PAST SALE,2024-12-20,Condo/Co-op,251 Crandon Blvd #1034,Key Biscayne,FL,33149.0,1825000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/251-Cra...,MARMLS,A11694679,N,Y,25.697697,-80.158296
103,PAST SALE,2024-12-20,Condo/Co-op,18001 Collins Ave Unit 2414+2415,Sunny Isles Beach,FL,33160.0,1030000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11579666,N,Y,25.943197,-80.120613
30,PAST SALE,2024-12-20,Condo/Co-op,9801 Collins Ave Unit 6G,Bal Harbour,FL,33154.0,1000000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/9801-Col...,MARMLS,A11690331,N,Y,25.890231,-80.122819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,PAST SALE,2024-12-16,Condo/Co-op,700 NE 26th Ter #1801,Miami,FL,33137.0,2700000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11569888,N,Y,25.802198,-80.185790
26,PAST SALE,2024-12-16,Condo/Co-op,1620 West Ave #406,Miami Beach,FL,33139.0,240000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1620-Wes...,MARMLS,A11690324,N,Y,25.789678,-80.142837
24,PAST SALE,2024-12-16,Condo/Co-op,8600 SW 109th Ave Unit 4-207,Miami,FL,33173.0,235000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/8600-SW-109th-...,MARMLS,A11668678,N,Y,25.688546,-80.370705
118,PAST SALE,2024-12-16,Condo/Co-op,19224 NE 25th Ave Unit 262D,Miami,FL,33180.0,413000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/19224-NE-25th-...,MARMLS,A11602684,N,Y,25.953645,-80.151856


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

151


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$961,594


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$620


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$145,200,758


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
102
Input Previous Week Condo Average Sales Price:
846364
Input Previous Week Condo Average PSF:
564
Input Previous Week Condo Sales total (ex: 198_000_000)
102_200_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-16,Condo/Co-op,9001 Collins Ave Unit S-1009,Surfside,FL,33154.0,14300000.0,2.0,4.5,SURF CLUB CONDO,2729.0,NaN,2017.0,NaN,5240.0,18650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11632519,N,Y,25.876848,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-16,Condo/Co-op,9001 Collins Ave Unit S-1009,Surfside,FL,33154.0,14300000.0,2.0,4.5,SURF CLUB CONDO,2729.0,NaN,2017.0,NaN,5240.0,18650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11632519,N,Y,25.876848,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange
1,PAST SALE,2024-12-16,Condo/Co-op,18501 Collins Ave #4901,Sunny Isles Beach,FL,33160.0,7600000.0,4.0,5.5,Turnberry Ocean Club,3625.0,NaN,2020.0,NaN,2097.0,6063.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11573386,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Devin Kay,Douglas Elliman,None,None,Haydar Alchalaby,"Compass Florida, LLC",None,None,2,blue
2,PAST SALE,2024-12-19,Condo/Co-op,68 SE 6th St Ph 4303,Miami,FL,33131.0,5800000.0,4.0,4.0,REACH CONDO,3092.0,NaN,2016.0,NaN,1876.0,4537.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/68-SE-6th-St-3...,MARMLS,A11711930,N,Y,25.768232,-80.192374,https://www.redfin.com/FL/Miami/68-SE-6th-St-3...,Elba Hernandez,Coldwell Banker Realty,None,None,Amy Lindenmoyer,Brickell Condo Real Estate,None,None,3,blue
3,PAST SALE,2024-12-17,Condo/Co-op,3535 Hiawatha Ave #202,Miami,FL,33133.0,5350000.0,4.0,5.5,RESIDENCES AT VIZCAYA CON,4520.0,NaN,2011.0,NaN,1184.0,5107.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3535-Hiawatha-...,MARMLS,A11625194,N,Y,25.739807,-80.218916,https://www.redfin.com/FL/Miami/3535-Hiawatha-...,Jessica Wolf,Luxe Properties,None,None,Ivannia Herrera,Miami New Realty,None,None,4,blue
4,PAST SALE,2024-12-16,Condo/Co-op,18501 Collins Ave #703,Sunny Isles Beach,FL,33160.0,5100000.0,3.0,4.5,TURNBERRY OCEAN CLUB,3110.0,NaN,2020.0,NaN,1640.0,5214.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11471089,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Richard Goihman,Douglas Elliman,Ivonne Goihman,Douglas Elliman,Michael Light,Douglas Elliman,None,None,5,blue
5,PAST SALE,2024-12-16,Condo/Co-op,17121 Collins Ave #2701,Sunny Isles Beach,FL,33160.0,3600000.0,4.0,4.5,JADE OCEAN CONDO,2485.0,NaN,2009.0,NaN,1449.0,3669.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11626868,N,Y,25.934401,-80.121005,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Allan Filgueiras,AF Realty,None,None,Claudine De Niro,Dezer Platinum Realty LLC,None,None,6,blue
6,PAST SALE,2024-12-19,Condo/Co-op,19955 NE 38th Ct #2203,Aventura,FL,33180.0,3200000.0,4.0,5.0,TOWERS OF PORTO VITA-SOUT,3300.0,NaN,2000.0,NaN,970.0,5405.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11624506,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Karen Matluck,"Compass Florida, LLC",Valentina Gonzalez Villalba,"Compass Florida, LLC",Karen Matluck,"Compass Florida, LLC",None,None,7,blue
7,PAST SALE,2024-12-16,Condo/Co-op,700 NE 26th Ter #1801,Miami,FL,33137.0,2700000.0,3.0,4.5,Missoni Baia,2586.0,NaN,2023.0,NaN,1044.0,3833.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11569888,N,Y,25.802198,-80.185790,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,Damaso Medina,United Realty Group Inc,None,None,Ivan Chorney,"Compass Florida, LLC.",None,None,8,blue
8,PAST SALE,2024-12-20,Condo/Co-op,102 24th St #1113,Miami Beach,FL,33139.0,2375000.0,1.0,1.5,2399 COLLINS AVENUE,970.0,NaN,2015.0,NaN,2448.0,2619.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/102-24th...

In [29]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [30]:
m.save('index.html')

## Data snagger

In [31]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [32]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-12-16,Condo/Co-op,9001 Collins Ave Unit S-1009,Surfside,FL,33154.0,14300000.0,2.0,4.5,SURF CLUB CONDO,2729.0,NaN,2017.0,NaN,5240.0,18650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11632519,N,Y,25.876848,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_12232024


# CREATE TEMPLATE 

In [35]:
muni_set = set(df_top_ten['CITY'])

In [36]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [37]:
df_top_ten.reset_index(inplace=True,drop=True)

In [38]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [39]:
top_sale

'SURF CLUB CONDO at 9001 Collins Ave Unit S-1009 in Surfside'

In [40]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [41]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [42]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-16,Condo/Co-op,9001 Collins Ave Unit S-1009,Surfside,FL,33154.0,14300000.0,2.0,4.5,SURF CLUB CONDO,2729.0,NaN,2017.0,NaN,5240.0,18650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11632519,N,Y,25.876848,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange,SURF CLUB CONDO at 9001 Collins Ave Unit S-100...
1,PAST SALE,2024-12-16,Condo/Co-op,18501 Collins Ave #4901,Sunny Isles Beach,FL,33160.0,7600000.0,4.0,5.5,Turnberry Ocean Club,3625.0,NaN,2020.0,NaN,2097.0,6063.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11573386,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Devin Kay,Douglas Elliman,None,None,Haydar Alchalaby,"Compass Florida, LLC",None,None,2,blue,Turnberry Ocean Club at 18501 Collins Ave #490...
2,PAST SALE,2024-12-19,Condo/Co-op,68 SE 6th St Ph 4303,Miami,FL,33131.0,5800000.0,4.0,4.0,REACH CONDO,3092.0,NaN,2016.0,NaN,1876.0,4537.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/68-SE-6th-St-3...,MARMLS,A11711930,N,Y,25.768232,-80.192374,https://www.redfin.com/FL/Miami/68-SE-6th-St-3...,Elba Hernandez,Coldwell Banker Realty,None,None,Amy Lindenmoyer,Brickell Condo Real Estate,None,None,3,blue,REACH CONDO at 68 SE 6th St Ph 4303 in Miami
3,PAST SALE,2024-12-17,Condo/Co-op,3535 Hiawatha Ave #202,Miami,FL,33133.0,5350000.0,4.0,5.5,RESIDENCES AT VIZCAYA CON,4520.0,NaN,2011.0,NaN,1184.0,5107.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3535-Hiawatha-...,MARMLS,A11625194,N,Y,25.739807,-80.218916,https://www.redfin.com/FL/Miami/3535-Hiawatha-...,Jessica Wolf,Luxe Properties,None,None,Ivannia Herrera,Miami New Realty,None,None,4,blue,RESIDENCES AT VIZCAYA CON at 3535 Hiawatha Ave...
4,PAST SALE,2024-12-16,Condo/Co-op,18501 Collins Ave #703,Sunny Isles Beach,FL,33160.0,5100000.0,3.0,4.5,TURNBERRY OCEAN CLUB,3110.0,NaN,2020.0,NaN,1640.0,5214.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11471089,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Richard Goihman,Douglas Elliman,Ivonne Goihman,Douglas Elliman,Michael Light,Douglas Elliman,None,None,5,blue,TURNBERRY OCEAN CLUB at 18501 Collins Ave #703...
5,PAST SALE,2024-12-16,Condo/Co-op,17121 Collins Ave #2701,Sunny Isles Beach,FL,33160.0,3600000.0,4.0,4.5,JADE OCEAN CONDO,2485.0,NaN,2009.0,NaN,1449.0,3669.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11626868,N,Y,25.934401,-80.121005,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Allan Filgueiras,AF Realty,None,None,Claudine De Niro,Dezer Platinum Realty LLC,None,None,6,blue,JADE OCEAN CONDO at 17121 Collins Ave #2701 in...
6,PAST SALE,2024-12-19,Condo/Co-op,19955 NE 38th Ct #2203,Aventura,FL,33180.0,3200000.0,4.0,5.0,TOWERS OF PORTO VITA-SOUT,3300.0,NaN,2000.0,NaN,970.0,5405.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11624506,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Karen Matluck,"Compass Florida, LLC",Valentina Gonzalez Villalba,"Compass Florida, LLC",Karen Matluck,"Compass Florida, LLC",None,None,7,blue,TOWERS OF PORTO VITA-SOUT at 19955 NE 38th Ct ...
7,PAST SALE,2024-12-16,Condo/Co-op,700 NE 26th Ter #1801,Miami,FL,33137.0,2700000.0,3.0,4.5,Missoni Baia,2586.0,NaN,2023.0,NaN,1044.0,3833.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11569888,N,Y,25.802198,-80.185790,https://

In [43]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: SURF CLUB CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,350,000M to $14,300,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Aventura, Miami, Sunny Isles Beach, Miami Beach, Surfside

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 151 condo sales totaling $145,200,758 million from Dec. 15th to Dec. 22nd. The previous week, brokers closed 102 condo sales totaling $102,200,000.

Last week’s units sold for an average of $961,594, higher than the $846,364 average price from the previous week.

In [44]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [45]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [46]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [47]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-16,Condo/Co-op,9001 Collins Ave Unit S-1009,Surfside,FL,33154.0,14300000.0,2.0,4.5,SURF CLUB CONDO,2729.0,NaN,2017.0,NaN,5240.0,18650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11632519,N,Y,25.876848,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange,SURF CLUB CONDO at 9001 Collins Ave Unit S-100...
1,PAST SALE,2024-12-16,Condo/Co-op,18501 Collins Ave #4901,Sunny Isles Beach,FL,33160.0,7600000.0,4.0,5.5,Turnberry Ocean Club,3625.0,NaN,2020.0,NaN,2097.0,6063.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11573386,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Devin Kay,Douglas Elliman,None,None,Haydar Alchalaby,"Compass Florida, LLC",None,None,2,blue,Turnberry Ocean Club at 18501 Collins Ave #490...
2,PAST SALE,2024-12-19,Condo/Co-op,68 SE 6th St Ph 4303,Miami,FL,33131.0,5800000.0,4.0,4.0,REACH CONDO,3092.0,NaN,2016.0,NaN,1876.0,4537.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/68-SE-6th-St-3...,MARMLS,A11711930,N,Y,25.768232,-80.192374,https://www.redfin.com/FL/Miami/68-SE-6th-St-3...,Elba Hernandez,Coldwell Banker Realty,None,None,Amy Lindenmoyer,Brickell Condo Real Estate,None,None,3,blue,REACH CONDO at 68 SE 6th St Ph 4303 in Miami
3,PAST SALE,2024-12-17,Condo/Co-op,3535 Hiawatha Ave #202,Miami,FL,33133.0,5350000.0,4.0,5.5,RESIDENCES AT VIZCAYA CON,4520.0,NaN,2011.0,NaN,1184.0,5107.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3535-Hiawatha-...,MARMLS,A11625194,N,Y,25.739807,-80.218916,https://www.redfin.com/FL/Miami/3535-Hiawatha-...,Jessica Wolf,Luxe Properties,None,None,Ivannia Herrera,Miami New Realty,None,None,4,blue,RESIDENCES AT VIZCAYA CON at 3535 Hiawatha Ave...
4,PAST SALE,2024-12-16,Condo/Co-op,18501 Collins Ave #703,Sunny Isles Beach,FL,33160.0,5100000.0,3.0,4.5,TURNBERRY OCEAN CLUB,3110.0,NaN,2020.0,NaN,1640.0,5214.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11471089,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Richard Goihman,Douglas Elliman,Ivonne Goihman,Douglas Elliman,Michael Light,Douglas Elliman,None,None,5,blue,TURNBERRY OCEAN CLUB at 18501 Collins Ave #703...
5,PAST SALE,2024-12-16,Condo/Co-op,17121 Collins Ave #2701,Sunny Isles Beach,FL,33160.0,3600000.0,4.0,4.5,JADE OCEAN CONDO,2485.0,NaN,2009.0,NaN,1449.0,3669.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11626868,N,Y,25.934401,-80.121005,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Allan Filgueiras,AF Realty,None,None,Claudine De Niro,Dezer Platinum Realty LLC,None,None,6,blue,JADE OCEAN CONDO at 17121 Collins Ave #2701 in...
6,PAST SALE,2024-12-19,Condo/Co-op,19955 NE 38th Ct #2203,Aventura,FL,33180.0,3200000.0,4.0,5.0,TOWERS OF PORTO VITA-SOUT,3300.0,NaN,2000.0,NaN,970.0,5405.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11624506,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Karen Matluck,"Compass Florida, LLC",Valentina Gonzalez Villalba,"Compass Florida, LLC",Karen Matluck,"Compass Florida, LLC",None,None,7,blue,TOWERS OF PORTO VITA-SOUT at 19955 NE 38th Ct ...
7,PAST SALE,2024-12-16,Condo/Co-op,700 NE 26th Ter #1801,Miami,FL,33137.0,2700000.0,3.0,4.5,Missoni Baia,2586.0,NaN,2023.0,NaN,1044.0,3833.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11569888,N,Y,25.802198,-80.185790,https://

In [48]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: SURF CLUB CONDO closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,350,000M to $14,300,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Aventura, Miami, Sunny Isles Beach, Miami Beach, Surfside

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 151 condo sales totaling $145,200,758 million from Dec. 15th to Dec. 22nd. The previous week, brokers closed 102 condo sales totaling $102,200,000.

Last week’s units sold for an average of $961,594, higher than the $846,364 average price from the previous week.

In [49]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Surfside/9001-Collins-Ave-33154/unit-S1009/home/79133184


In [50]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Sunny-Isles-Beach/18501-Collins-Ave-33160/unit-4901/home/175110931


In [51]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/500-Brickell-Ave-33131/unit-4/home/43399253


In [52]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami-Beach/102-24th-St-33139/unit-1113/home/79136198


In [53]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Surfside/9001-Collins-Ave-33154/unit-S1009/home/79133184


In [54]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami/500-Brickell-Ave-33131/unit-4/home/43399253


In [55]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-12-16,Condo/Co-op,9001 Collins Ave Unit S-1009,Surfside,FL,33154.0,14300000.0,2.0,4.5,SURF CLUB CONDO,2729.0,NaN,2017.0,NaN,5240.0,18650.0,Sold,NaN,NaN,https://www.redfin.com/FL/Surfside/9001-Collin...,MARMLS,A11632519,N,Y,25.876848,-80.121466,https://www.redfin.com/FL/Surfside/9001-Collin...,Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,Ximena Penuela,"Fort Realty, LLC",None,None,1,orange,SURF CLUB CONDO at 9001 Collins Ave Unit S-100...
1,PAST SALE,2024-12-16,Condo/Co-op,18501 Collins Ave #4901,Sunny Isles Beach,FL,33160.0,7600000.0,4.0,5.5,Turnberry Ocean Club,3625.0,NaN,2020.0,NaN,2097.0,6063.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11573386,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Devin Kay,Douglas Elliman,None,None,Haydar Alchalaby,"Compass Florida, LLC",None,None,2,blue,Turnberry Ocean Club at 18501 Collins Ave #490...
2,PAST SALE,2024-12-19,Condo/Co-op,68 SE 6th St Ph 4303,Miami,FL,33131.0,5800000.0,4.0,4.0,REACH CONDO,3092.0,NaN,2016.0,NaN,1876.0,4537.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/68-SE-6th-St-3...,MARMLS,A11711930,N,Y,25.768232,-80.192374,https://www.redfin.com/FL/Miami/68-SE-6th-St-3...,Elba Hernandez,Coldwell Banker Realty,None,None,Amy Lindenmoyer,Brickell Condo Real Estate,None,None,3,blue,REACH CONDO at 68 SE 6th St Ph 4303 in Miami
3,PAST SALE,2024-12-17,Condo/Co-op,3535 Hiawatha Ave #202,Miami,FL,33133.0,5350000.0,4.0,5.5,RESIDENCES AT VIZCAYA CON,4520.0,NaN,2011.0,NaN,1184.0,5107.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3535-Hiawatha-...,MARMLS,A11625194,N,Y,25.739807,-80.218916,https://www.redfin.com/FL/Miami/3535-Hiawatha-...,Jessica Wolf,Luxe Properties,None,None,Ivannia Herrera,Miami New Realty,None,None,4,blue,RESIDENCES AT VIZCAYA CON at 3535 Hiawatha Ave...
4,PAST SALE,2024-12-16,Condo/Co-op,18501 Collins Ave #703,Sunny Isles Beach,FL,33160.0,5100000.0,3.0,4.5,TURNBERRY OCEAN CLUB,3110.0,NaN,2020.0,NaN,1640.0,5214.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11471089,N,Y,25.947720,-80.120228,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,Richard Goihman,Douglas Elliman,Ivonne Goihman,Douglas Elliman,Michael Light,Douglas Elliman,None,None,5,blue,TURNBERRY OCEAN CLUB at 18501 Collins Ave #703...
5,PAST SALE,2024-12-16,Condo/Co-op,17121 Collins Ave #2701,Sunny Isles Beach,FL,33160.0,3600000.0,4.0,4.5,JADE OCEAN CONDO,2485.0,NaN,2009.0,NaN,1449.0,3669.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11626868,N,Y,25.934401,-80.121005,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Allan Filgueiras,AF Realty,None,None,Claudine De Niro,Dezer Platinum Realty LLC,None,None,6,blue,JADE OCEAN CONDO at 17121 Collins Ave #2701 in...
6,PAST SALE,2024-12-19,Condo/Co-op,19955 NE 38th Ct #2203,Aventura,FL,33180.0,3200000.0,4.0,5.0,TOWERS OF PORTO VITA-SOUT,3300.0,NaN,2000.0,NaN,970.0,5405.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11624506,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Karen Matluck,"Compass Florida, LLC",Valentina Gonzalez Villalba,"Compass Florida, LLC",Karen Matluck,"Compass Florida, LLC",None,None,7,blue,TOWERS OF PORTO VITA-SOUT at 19955 NE 38th Ct ...
7,PAST SALE,2024-12-16,Condo/Co-op,700 NE 26th Ter #1801,Miami,FL,33137.0,2700000.0,3.0,4.5,Missoni Baia,2586.0,NaN,2023.0,NaN,1044.0,3833.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/700-NE-26th-Te...,MARMLS,A11569888,N,Y,25.802198,-80.185790,https://

## Time on Market Calculator

In [67]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 4, 22) ## List (Earlier) date
date2 = datetime(2024, 12, 16) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

238


## Clean CSV for Datawrapper Chart

In [56]:
chart_df = df_top_ten

In [57]:
chart_df = chart_df.fillna(" ")

In [58]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [59]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [60]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [61]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [62]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [63]:
csv_date_string = today.strftime("%m_%d_%Y")

In [64]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [65]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-12-16,9001 Collins Ave Unit S-1009 in Surfside,"$14,300,000",2,4.5,"2,729",2017,"$5,240",MARMLS,A11632519,Michael Duchon The Corcoran Group Eloy Carmena...,"Ximena Penuela Fort Realty, LLC"
1,2024-12-16,18501 Collins Ave #4901 in Sunny Isles Beach,"$7,600,000",4,5.5,"3,625",2020,"$2,097",MARMLS,A11573386,Devin Kay Douglas Elliman,"Haydar Alchalaby Compass Florida, LLC"
2,2024-12-19,68 SE 6th St Ph 4303 in Miami,"$5,800,000",4,4.0,"3,092",2016,"$1,876",MARMLS,A11711930,Elba Hernandez Coldwell Banker Realty,Amy Lindenmoyer Brickell Condo Real Estate
3,2024-12-17,3535 Hiawatha Ave #202 in Miami,"$5,350,000",4,5.5,"4,520",2011,"$1,184",MARMLS,A11625194,Jessica Wolf Luxe Properties,Ivannia Herrera Miami New Realty
4,2024-12-16,18501 Collins Ave #703 in Sunny Isles Beach,"$5,100,000",3,4.5,"3,110",2020,"$1,640",MARMLS,A11471089,Richard Goihman Douglas Elliman Ivonne Goihman...,Michael Light Douglas Elliman
5,2024-12-16,17121 Collins Ave #2701 in Sunny Isles Beach,"$3,600,000",4,4.5,"2,485",2009,"$1,449",MARMLS,A11626868,Allan Filgueiras AF Realty,Claudine De Niro Dezer Platinum Realty LLC
6,2024-12-19,19955 NE 38th Ct #2203 in Aventura,"$3,200,000",4,5.0,"3,300",2000,$970,MARMLS,A11624506,"Karen Matluck Compass Florida, LLC Valentina G...","Karen Matluck Compass Florida, LLC"
7,2024-12-16,700 NE 26th Ter #1801 in Miami,"$2,700,000",3,4.5,"2,586",2023,"$1,044",MARMLS,A11569888,Damaso Medina United Realty Group Inc,"Ivan Chorney Compass Florida, LLC."
8,2024-12-20,102 24th St #1113 in Miami Beach,"$2,375,000",1,1.5,970,2015,"$2,448",MARMLS,A11626709,Tracy Galya Douglas Elliman,"Jennifer Harley Compass Florida, LLC"
9,2024-12-17,500 Brickell Ave Ph -4 in Miami,"$2,350,000",3,3.5,"3,800",2008,$618,MARMLS,A11329453,Daniel Glynn Elite Ocean View Realty LLC,Stefania Mogollon Ancona Real Estate Inc
